In [1]:
import importlib.metadata
import sys
import platform
import warnings

# 경고 메시지 잠시 끄기 (버전 확인에 집중)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# 사용된 코드에서 import 되었거나, 필요할 것으로 예상되는 패키지 목록
packages_to_check = [
    "pandas",
    "sentence-transformers",
    "langchain",
    "langchain-huggingface", # Deprecation Warning 관련 새 패키지
    "chromadb",
    "transformers",
    "bitsandbytes",          # BitsAndBytesConfig 사용 시 필요
    "peft",                  # PeftModel 사용 시 필요
    "torch",                 # transformers, sentence-transformers 등에서 주로 사용
    "accelerate",            # device_map='auto', 양자화 등에 필요할 수 있음
    "pysqlite3-binary",      # ChromaDB sqlite3 문제 해결용
]

print(f"--- System/Python Info ---")
print(f"Python Version: {platform.python_version()}")
print(f"Platform: {platform.system()} {platform.release()} ({platform.machine()})")
print(f"--------------------------")
print(f"--- Required Package Versions ---")

installed_versions = {}
for pkg in packages_to_check:
    try:
        # importlib.metadata를 사용하여 패키지 버전 가져오기
        version = importlib.metadata.version(pkg)
        installed_versions[pkg] = version
        print(f"{pkg}: {version}")
    except importlib.metadata.PackageNotFoundError:
        # 패키지가 설치되지 않은 경우
        installed_versions[pkg] = "Not Found"
        print(f"{pkg}: Not Found")
    except Exception as e:
        # 다른 오류 발생 시
        installed_versions[pkg] = f"Error ({e})"
        print(f"{pkg}: Error checking version ({e})")

# 파이썬이 현재 사용 중인 sqlite3 버전 확인
print(f"--------------------------")
print(f"--- Active SQLite3 Info ---")
try:
    # pysqlite3 오버라이드가 적용되었는지 확인하기 위해 sqlite3를 import
    # 주의: 이 코드 실행 시점에 오버라이드 코드가 먼저 실행되지 않았다면 기본 버전이 나올 수 있음
    import sqlite3
    active_sqlite_version = sqlite3.sqlite_version

    # pysqlite3-binary가 활성화된 모듈인지 감지 시도
    is_pysqlite_active = False
    if 'pysqlite3' in sys.modules:
         # 실제 사용중인 sqlite3 모듈이 pysqlite3 모듈과 같은지 확인
         if 'sqlite3' in sys.modules and sys.modules['sqlite3'] == sys.modules['pysqlite3']:
             is_pysqlite_active = True
         # 또는 pysqlite3 에서만 제공하는 속성이 있는지 확인 (간접적인 방법)
         elif hasattr(sqlite3, '_pysqlite_version'):
             is_pysqlite_active = True

    print(f"Active SQLite3 Version Used by Python: {active_sqlite_version}")
    if is_pysqlite_active:
         print("(Note: Actively using sqlite3 from pysqlite3-binary)")
    else:
         # 표준 라이브러리를 사용하지만, pysqlite3-binary가 설치는 되어있는지 확인
         if installed_versions.get("pysqlite3-binary") != "Not Found":
             print("(Note: Using standard library sqlite3, but pysqlite3-binary is installed)")
         else:
             print("(Note: Using standard library sqlite3, pysqlite3-binary not found/installed)")

except Exception as e:
    print(f"Could not check active sqlite3 version: {e}")
print(f"--------------------------")

# 원래 경고 설정 복원 (필요시)
# warnings.resetwarnings()

--- System/Python Info ---
Python Version: 3.8.10
Platform: Linux 5.4.0-150-generic (x86_64)
--------------------------
--- Required Package Versions ---
pandas: 2.0.3
sentence-transformers: 3.2.1
langchain: 0.2.17
langchain-huggingface: Not Found
chromadb: 0.5.23
transformers: 4.46.3
bitsandbytes: 0.42.0
peft: 0.13.2
torch: 2.4.1
accelerate: 1.0.1
pysqlite3-binary: Not Found
--------------------------
--- Active SQLite3 Info ---
Active SQLite3 Version Used by Python: 3.31.1
(Note: Using standard library sqlite3, pysqlite3-binary not found/installed)
--------------------------
